# core

> Functions that are not specific to individual data sources

These functions are available directly from the finsets namespace. For example to use the `features_metadata` function below, use 

```python
from finsets import features_metadata
```


In [ ]:
#| default_exp core

In [ ]:
#| export
from __future__ import annotations
from typing import Literal
from importlib import import_module
from inspect import signature

import pandas as pd
from  thefuzz import process, fuzz

from fastcore.script import call_parse

In [ ]:
#| export
def features_metadata(submodules: list=['wrds', 'papers'] # list of submodules to collect metadata from
                      ) -> pd.DataFrame:
    "Go through `submodules` of `finsets` and collect metadata from all functions that have `return_metadata` parameter"
    
    df = pd.DataFrame(columns=['name','label','output_of','inputs','inputs_generated_by'])
    for name in submodules:
        module = import_module(f'finsets.{name}')
        for sub in dir(module):
            if sub.startswith('_'): continue
            submodule = import_module(f'finsets.{name}.{sub}')
            for func_name in submodule.__all__:
                func = getattr(submodule, func_name)
                if callable(func):
                    try: 
                        params = signature(func).parameters
                    except:
                        continue
                    if 'return_metadata' in params: 
                        meta = func(return_metadata=True)
                        for var_name in meta['outputs']:
                            for input_name in meta['inputs']:    
                                new_meta = pd.DataFrame({'name':var_name, 
                                                'label':meta['labels'][var_name], 
                                                'output_of':f'{name}.{sub}.{func_name}', 
                                                'inputs':','.join(meta['inputs'][input_name]),
                                                'inputs_generated_by':input_name}, index=[0])
                                df = pd.concat([df,new_meta],ignore_index=True)
    return df

In [ ]:
features_metadata()

,name,label,output_of,inputs,inputs_generated_by
0,bookeq,Book equity,wrds.compa.book_equity,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb",wrds.compa.clean
1,shreq,Shareholder equity,wrds.compa.book_equity,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb",wrds.compa.clean
2,pref_stock,Preferred stock,wrds.compa.book_equity,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb",wrds.compa.clean
3,tobinq,Tobin Q,wrds.compa.tobin_q,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb,pr...",wrds.compa.clean
4,equityiss_tot,Equity issuance,wrds.compa.issuance_vars,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb,ss...",wrds.compa.clean
5,equityiss_cfs,Equity issuance,wrds.compa.issuance_vars,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb,ss...",wrds.compa.clean
6,debtiss_tot,Debt issuance,wrds.compa.issuance_vars,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb,ss...",wrds.compa.clean
7,debtiss_cfs,Debt issuance,wrds.compa.issuance_vars,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb,ss...",wrds.compa.clean
8,debtiss_bs,Debt issuance,wrds.compa.issuance_vars,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb,ss...",wrds.compa.clean
9,ppentpch,Pct change in net PPE,wrds.compa.investment_vars,"ppent,capx,at",wrds.compa.clean


In [ ]:
#| export
def raw_metadata(submodules=['wrds', 'papers'] # list of submodules to collect metadata from
                ) -> pd.DataFrame:
    "Go through `submodules` of `finsets` and collect metadata from `raw_metadata` functions (if present)"

    df = pd.DataFrame(columns=['name','label','output_of','type'])
    for name in submodules:
        module = import_module(f'finsets.{name}')
        for sub in dir(module):
            if sub.startswith('_'): continue
            submodule = import_module(f'finsets.{name}.{sub}')
            if 'raw_metadata' in submodule.__all__:
                df = pd.concat([df,submodule.raw_metadata()],ignore_index=True)
    return df

In [ ]:
raw_metadata()

,name,label,output_of,type
0,gvkey,Global Company Key,wrds.compa.clean,string
1,conm,Company Name,wrds.compa.clean,string
2,tic,Ticker Symbol,wrds.compa.clean,string
3,cusip,CUSIP,wrds.compa.clean,string
4,cik,CIK Number,wrds.compa.clean,string
...,...,...,...,...
1031,vwretd,Value-Weighted Return (includes distributions),wrds.crspm.clean,double
1032,vwretx,Value-Weighted Return (excluding dividends),wrds.crspm.clean,double
1033,ewretd,Equal-Weighted Return (includes distributions),wrds.crspm.clean,double
1034,ewretx,Equal-Weighted Return (excluding dividends),wrds.crspm.clean,double


In [ ]:
#| export
def all_metadata(submodules=['wrds', 'papers'] # list of submodules to collect metadata from
                ) -> pd.DataFrame:
    "Collects `raw_metadata` and `features_metadata` from `submodules` of `finsets`"

    return pd.concat([features_metadata(submodules), raw_metadata(submodules)], ignore_index=True)

In [ ]:
all_metadata()

,name,label,output_of,inputs,inputs_generated_by,type
0,bookeq,Book equity,wrds.compa.book_equity,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb",wrds.compa.clean,NaN
1,shreq,Shareholder equity,wrds.compa.book_equity,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb",wrds.compa.clean,NaN
2,pref_stock,Preferred stock,wrds.compa.book_equity,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb",wrds.compa.clean,NaN
3,tobinq,Tobin Q,wrds.compa.tobin_q,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb,pr...",wrds.compa.clean,NaN
4,equityiss_tot,Equity issuance,wrds.compa.issuance_vars,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb,ss...",wrds.compa.clean,NaN
...,...,...,...,...,...,...
1050,vwretd,Value-Weighted Return (includes distributions),wrds.crspm.clean,NaN,NaN,double
1051,vwretx,Value-Weighted Return (excluding dividends),wrds.crspm.clean,NaN,NaN,double
1052,ewretd,Equal-Weighted Return (includes distributions),wrds.crspm.clean,NaN,NaN,double
1053,ewretx,Equal-Weighted Return (excluding dividends),wrds.crspm.clean,NaN,NaN,double


In [ ]:
#| export
@call_parse
def search(query: str,              # What to search for 
           meta: str='all',   #"all", "features", or "raw"; specifies the function that fetches the metadata you want to search through
           field: str='label', # Which column in the metadata table you want to search through. Use "name" to search variable names.  
           limit: int=10,           # How many results to display                      
           ) -> pd.DataFrame:
    "Search for `query` in metadata returned by f`{meta_func}_metadata`; return `limit` number of results"

    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 100)

    try:   
        import finsets
        metadata = getattr(finsets, f'{meta}_metadata')()
        results = process.extractBests(query, metadata[field], 
                                        scorer = fuzz.token_sort_ratio,
                                        limit=limit)
        rows = [x[2] for x in results]
        scores = [x[1] for x in results]
        df = metadata.iloc[rows]
        df.index = scores
        df.index.name="SCORE"
        df.columns = df.columns.str.upper()
    finally:
        pd.reset_option('display.max_columns')
        pd.reset_option('display.width')        
    return df
       

In [ ]:
search('tobins q')

,NAME,LABEL,OUTPUT_OF,INPUTS,INPUTS_GENERATED_BY,TYPE
SCORE,,,,,,
93,tobinq,Tobin Q,wrds.compa.tobin_q,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb,pr...",wrds.compa.clean,NaN
50,aqc,Acquisitions,wrds.compa.clean,NaN,NaN,double
45,acqlntal,Acquired Loans,wrds.compa.clean,NaN,NaN,double
43,pvon,Provisions - Other (Net),wrds.compa.clean,NaN,NaN,double
43,optgr,Options - Granted,wrds.compa.clean,NaN,NaN,double
42,gsector,GIC Sectors,wrds.compa.clean,NaN,NaN,string
42,bid,Closing Bid,wrds.crspm.clean,NaN,NaN,double
42,ask,Closing Ask,wrds.crspm.clean,NaN,NaN,double
42,trtscd,Traits Code,wrds.crspm.clean,NaN,NaN,double


In [ ]:
search('total assets', meta = 'raw', limit=5)

,NAME,LABEL,OUTPUT_OF,TYPE
SCORE,,,,
100,at,Assets - Total,wrds.compa.clean,double
75,act,Current Assets - Total,wrds.compa.clean,double
75,ao,Assets - Other,wrds.compa.clean,double
71,batr,Benefits Assumed - Total,wrds.compa.clean,double
69,dptb,Deposits - Total - Banks,wrds.compa.clean,double


In [ ]:
search('cash flows', meta = 'features')

,NAME,LABEL,OUTPUT_OF,INPUTS,INPUTS_GENERATED_BY
SCORE,,,,,
54,cflow2la_is,Cash flows to lagged assets,wrds.compa.cashflow_vars,"dtdate,oancf,ib,dp,at",wrds.compa.clean
54,cflow2la_cfs,Cash flows to lagged assets,wrds.compa.cashflow_vars,"dtdate,oancf,ib,dp,at",wrds.compa.clean
54,cflow2la_full,Cash flows to lagged assets,wrds.compa.cashflow_vars,"dtdate,oancf,ib,dp,at",wrds.compa.clean
42,cash2a,Cash holdings to assets,wrds.compa.liquidity_vars,"che,at",wrds.compa.clean
32,capx2la,CAPX to lagged assets,wrds.compa.investment_vars,"ppent,capx,at",wrds.compa.clean
31,roa,Return on assets,wrds.compa.profitability_vars,"ib,at",wrds.compa.clean
28,div2la,Dividends to lagged assets,wrds.compa.payout_vars,"dvc,prstkc,at",wrds.compa.clean
26,rep2la,Repurchases to lagged assets,wrds.compa.payout_vars,"dvc,prstkc,at",wrds.compa.clean
24,tobinq,Tobin Q,wrds.compa.tobin_q,"at,lt,seq,ceq,txditc,pstk,pstkrv,pstkl,itcb,pr...",wrds.compa.clean


In [ ]:
search('txditc', field='name', limit=5)

,NAME,LABEL,OUTPUT_OF,INPUTS,INPUTS_GENERATED_BY,TYPE
SCORE,,,,,,
100,txditc,Deferred Taxes and Investment Tax Credit,wrds.compa.clean,NaN,NaN,double
80,txdi,Income Taxes - Deferred,wrds.compa.clean,NaN,NaN,double
80,txdc,Deferred Taxes (Cash Flow),wrds.compa.clean,NaN,NaN,double
67,tic,Ticker Symbol,wrds.compa.clean,NaN,NaN,string
67,txdbca,Deferred Tax Asset - Current,wrds.compa.clean,NaN,NaN,double


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()